In [ ]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole



* take iupac names and generate dataset with smiles, mp, bp, etc

* create mol objects with smiles

* calculate autocorr2D descriptors
* add them to the dataframe

* set up ML algorithm to predict mp or bp using autocorr2D

* view the results, determine best descriptors